In [1]:
%pylab inline
import os
import gc
import scipy
import healpy as hp
import numpy as np
import astropy.io.fits as pyfits
import astropy.table as astTable
import catutil as utilities
import imgSimutil as imgUtil

Populating the interactive namespace from numpy and matplotlib


In [2]:
cd wrkDir/FPFS2/sim/

/lustre/work/xiangchong.li/work/FPFS2/sim


In [4]:
igroup      =   1744588
ngrid       =   64
ngridT      =   ngrid*100
noiVar      =   3.6e-3
opend       =   'var36em4'
pixScale    =   0.168
psfFWHM     =   '60'
psfFWHMF    =   eval(psfFWHM)/100.

# necessary directories
galDir      =   'galaxy_cosmo_psf%s' %psfFWHM
galDir2     =   'galaxy_cosmoE_psf%s' %psfFWHM
noiDir      =   'noise'
assert os.path.exists(galDir)
assert os.path.exists(galDir2)
assert os.path.exists(noiDir)

# noise
ifield      =   1
noiFname    =   os.path.join(noiDir,'noi%04d.fits' %ifield)
# multiply by 10 since the noise has variance 0.01
noiData     =   pyfits.getdata(noiFname)*10.

In [7]:
ishear      =   'g1-0000'#,'g1-0002','g1-0020','g1-0200','g1-2000','g1-2222']
for pixId in imgUtil.cosmoHSThpix:
    galFname    =   os.path.join(galDir,'image-%s-%s.fits' %(igroup,ishear))
    galData     =   pyfits.getdata(galFname)
    galFname2   =   os.path.join(galDir2,'image-%s-%s.fits' %(igroup,ishear))
    galData2    =   pyfits.getdata(galFname2)
    assert galData2.shape==galData.shape

In [6]:
# PSF
psfFname    =   os.path.join(galDir,'psf-%s.fits' %psfFWHM)
psfData     =   pyfits.getdata(psfFname)
npad        =   (ngrid-psfData.shape[0])//2
psfData2    =   np.pad(psfData,(npad+1,npad),mode='constant')
assert psfData2.shape[0]==ngrid

In [12]:
ishear      =   'g1-0000'#,'g1-0002','g1-0020','g1-0200','g1-2000','g1-2222']
galFname    =   os.path.join(galDir,'image-%s-%s.fits' %(igroup,ishear))
galData     =   pyfits.getdata(galFname)
ny,nx       =   galData.shape
galData     =   galData+noiData[0:ny,0:nx]*np.sqrt(noiVar)
#exposure    =   makeHSCExposure(galData,psfData,pixScale,noiVar)

In [ ]:
def makeHSCExposure(galData,psfData,pixScale,variance):
    ny,nx=galData.shape
    exposure    =   afwImg.ExposureF(nx,ny)
    exposure.getMaskedImage().getImage().getArray()[:,:]=galData
    exposure.getMaskedImage().getVariance().getArray()[:,:]=variance
    #Set the PSF
    ngridPsf    =   psfData.shape[0]
    psfLsst     =   afwImg.ImageF(ngridPsf,ngridPsf)
    psfLsst.getArray()[:,:]= psfData
    psfLsst     =   psfLsst.convertD()
    kernel      =   afwMath.FixedKernel(psfLsst)
    kernelPSF   =   meaAlg.KernelPsf(kernel)
    exposure.setPsf(kernelPSF)
    #prepare the wcs
    #Rotation
    cdelt   =   (pixScale*afwGeom.arcseconds)
    CD      =   afwGeom.makeCdMatrix(cdelt, afwGeom.Angle(0.))#no rotation
    #wcs
    crval   =   afwGeom.SpherePoint(afwGeom.Angle(0.,afwGeom.degrees),afwGeom.Angle(0.,afwGeom.degrees))
    crpix   =   afwGeom.Point2D(0.0, 0.0)
    dataWcs =   afwGeom.makeSkyWcs(crpix,crval,CD)
    exposure.setWcs(dataWcs)
    #prepare the frc
    dataCalib = afwImg.makePhotoCalibFromCalibZeroPoint(63095734448.0194)
    exposure.setPhotoCalib(dataCalib)
    return exposure